## Trazer os dados das tabelas do banco de dados para DataFrames
Começo Trazendo os dados das tabelas do banco de dados para DataFrames

In [ ]:
import psycopg2
import pandas as pd
from dotenv import load_dotenv
import os

# Carregar variáveis do arquivo .env
load_dotenv()

# Pegar as credenciais do arquivo .env
db_host = os.getenv('DB_HOST')
db_database = os.getenv('DB_DATABASE')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_port = os.getenv('DB_PORT')

# Conectar ao banco de dados PostgreSQL usando as variáveis do .env
conn = psycopg2.connect(
    host=db_host,
    database=db_database,
    user=db_user,
    password=db_password,
    port=db_port
)

# Função para trazer os dados de uma tabela e convertê-los em DataFrame
def tabela_para_dataframe(tabela):
    query = f"SELECT * FROM {tabela}"
    return pd.read_sql(query, conn)

# Trazer os dados das tabelas atendimento, unidadeSaude, paciente e profissionalSaude
df_atendimentos = tabela_para_dataframe('atendimento')
df_unidade = tabela_para_dataframe('unidadeSaude')
df_paciente = tabela_para_dataframe('paciente')
df_profissional = tabela_para_dataframe('profissionalSaude')


In [ ]:
df_atendimentos

In [ ]:
df_paciente

In [ ]:

# Exibir os primeiros registros do DataFrame de atendimentos para validação
df_atendimentos.head(5)


In [ ]:
# Exibir informações dos DataFrames
print("Atendimentos:")
df_atendimentos.describe()


In [ ]:

print("\nUnidades de Saúde:")
df_unidade.info()


In [ ]:

print("\nPacientes:")
df_paciente.info()

print("\nProfissionais de Saúde:")
df_profissional.info()


### Preparando os dados para o agrupamento
Crie um data frame para cada agrupamento que desejo fazer as previsoes

In [25]:
# Converter a coluna data_hora_atendimento para datetime, se necessário
df_atendimentos['data_hora_atendimento'] = pd.to_datetime(df_atendimentos['data_hora_atendimento'])

# Agrupamento Diário
df_diario = df_atendimentos.groupby(df_atendimentos['data_hora_atendimento'].dt.date).size().reset_index(name='num_atendimentos_diarios')

# Agrupamento Semanal
df_semanal = df_atendimentos.groupby(df_atendimentos['data_hora_atendimento'].dt.to_period('W')).size().reset_index(name='num_atendimentos_semanal')

# Agrupamento Mensal
df_mensal = df_atendimentos.groupby(df_atendimentos['data_hora_atendimento'].dt.to_period('M')).size().reset_index(name='num_atendimentos_mensal')

# Exibir os primeiros resultados para validação
print("Atendimentos Diários:")
print(df_diario.head())

print("\nAtendimentos Semanais:")
print(df_semanal.head())

print("\nAtendimentos Mensais:")
print(df_mensal.head())


Atendimentos Diários:
  data_hora_atendimento  num_atendimentos_diarios
0            2018-01-01                        28
1            2018-01-02                        29
2            2018-01-03                        29
3            2018-01-04                        30
4            2018-01-05                        29

Atendimentos Semanais:
   data_hora_atendimento  num_atendimentos_semanal
0  2018-01-01/2018-01-07                       205
1  2018-01-08/2018-01-14                       206
2  2018-01-15/2018-01-21                       203
3  2018-01-22/2018-01-28                       205
4  2018-01-29/2018-02-04                       221

Atendimentos Mensais:
  data_hora_atendimento  num_atendimentos_mensal
0               2018-01                      906
1               2018-02                      941
2               2018-03                     1132
3               2018-04                     1164
4               2018-05                     1224


In [ ]:
# Exibir as primeiras linhas do agrupamento diário
df_diario.head()

# Visualizar os atendimentos diários
df_diario.plot(x='data_hora_atendimento', y='num_atendimentos_diarios', title='Atendimentos Diários', ylabel='Número de Atendimentos')


In [ ]:
df_diario.head()

In [28]:
df_diario['data_hora_atendimento'] = pd.to_datetime(df_diario['data_hora_atendimento'])

# Criar uma nova coluna 'dia_atendimento' contendo apenas a data no formato 'ano/mês/dia'
df_diario['dia_atendimento'] = df_diario['data_hora_atendimento'].dt.strftime('%d/%m/%Y')

# Ordenar o DataFrame pela nova coluna 'dia_atendimento'
df_diario = df_diario.sort_values(by='dia_atendimento')

# Exibir o DataFrame atualizado
print(df_diario)

    data_hora_atendimento  num_atendimentos_diarios   dia_atend  \
0              2018-01-01                        28  2018-01-01   
31             2018-02-01                        32  2018-02-01   
59             2018-03-01                        38  2018-03-01   
90             2018-04-01                        38  2018-04-01   
120            2018-05-01                        36  2018-05-01   
..                    ...                       ...         ...   
149            2018-05-30                        38  2018-05-30   
180            2018-06-30                        40  2018-06-30   
30             2018-01-31                        29  2018-01-31   
89             2018-03-31                        34  2018-03-31   
150            2018-05-31                        42  2018-05-31   

    dia_atendimento  
0        01/01/2018  
31       01/02/2018  
59       01/03/2018  
90       01/04/2018  
120      01/05/2018  
..              ...  
149      30/05/2018  
180      30/06/2018

In [26]:
# Exibir os primeiros registros do DataFrame com data sem as horas e sem barras
df_diario['data_hora_atendimento'] = pd.to_datetime(df_diario['data_hora_atendimento'])
df_diario['dia_atend'] = df_diario['data_hora_atendimento'].dt.date
df_diario = df_diario.sort_values(by='dia_atend')


In [27]:
df_diario

,data_hora_atendimento,num_atendimentos_diarios,dia_atend
0,2018-01-01,28,2018-01-01
1,2018-01-02,29,2018-01-02
2,2018-01-03,29,2018-01-03
3,2018-01-04,30,2018-01-04
4,2018-01-05,29,2018-01-05
...,...,...,...
176,2018-06-26,41,2018-06-26
177,2018-06-27,41,2018-06-27
178,2018-06-28,36,2018-06-28
179,2018-06-29,38,2018-06-29


In [ ]:
import matplotlib.pyplot as plt

# Gráfico de tendência dos atendimentos diários
plt.figure(figsize=(12, 6))
plt.plot(df_diario['data_hora_atendimento'], df_diario['num_atendimentos_diarios'], marker='o', linestyle='-', label='Diário')
plt.title('Tendência dos Atendimentos Diários')
plt.xlabel('Data')
plt.ylabel('Número de Atendimentos')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Gráfico de tendência dos atendimentos semanais
plt.figure(figsize=(36, 14))
plt.plot(df_semanal['data_hora_atendimento'].astype(str), df_semanal['num_atendimentos_semanal'], marker='o', linestyle='-', label='Semanal')
plt.title('Tendência dos Atendimentos Semanais')
plt.xlabel('Semana')
plt.ylabel('Número de Atendimentos')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Gráfico de tendência dos atendimentos mensais
plt.figure(figsize=(12, 6))
plt.plot(df_mensal['data_hora_atendimento'].astype(str), df_mensal['num_atendimentos_mensal'], marker='o', linestyle='-', label='Mensal')
plt.title('Tendência dos Atendimentos Mensais')
plt.xlabel('Mês')
plt.ylabel('Número de Atendimentos')
plt.legend()
plt.grid(True)
plt.show()

## DESAFIO: 

#### depois do dia 02/10/2024 vou modificar a base para diminuir a variacao
#### Tentar prever a demanda por bairro do paciente.

1. Juntar as tabelas: Precisamos unir os dados de atendimentos com os dados da tabela de pacientes, pois o bairro está armazenado nela.
2. Agrupar os atendimentos por bairro: Uma vez que os dados estejam unidos, podemos agrupar os atendimentos por bairro.
3. Preparar os dados para prever a demanda por bairro.
4. Lembrem de ordenar os dados de acordo com a coluna de tempo para realizar precisoes temporais.